# Counts per researcher

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make charts from `carcinos` database with focus on time and space exploration. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Planilha geral Atualizada FINAL 5_GERAL_sendo trabalhada no Google drive.xlsx'</font>


In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8', low_memory=False)

## Filtering

At least for now, we'll be considering only specimens of order decapoda (deeply revised by the Museum's crew)

In [3]:
decapoda = NewTable[NewTable['order'] == 'Decapoda'].copy()

<br>

<font size=5>**Color palette**</font>

Colors (per infraorder): 

- <font color='#e26d67'><b>Ascacidae</b></font>
- <font color='#007961'><b>Anomura</b></font>
- <font color='#7a2c39'><b>Achelata</b></font>
- <font color='#b67262'><b>Axiidea</b></font>
- <font color='#ee4454'><b>Brachyura</b></font>
- <font color='#3330b7'><b>Caridea</b></font>
- <font color='#58b5e1'><b>Gebiidea</b></font>
- <font color='#b8e450'><b>Stenopodídea</b></font>
- <font color='#a0a3fd'><b>Astacidae</b></font>
- <font color='#deae9e'><b>Polychelida</b></font>
- <font color='#d867be'><b>Grapsidae</b></font>
- <font color='#fece5f'><b>Xanthoidea</b></font>

In [4]:
# importing customized color palettes
from src.MNViz_colors import *

<br>


## Graphs

---

### counts per determiner per year

In [5]:
teste = decapoda.groupby(['determiner_full_name','det_year']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [6]:
# gráfico não tem todas as contagens por causa de nomes faltando
g1 = alt.Chart(teste, width=600, height=500).mark_circle().encode(
    x= alt.X('det_year', type='ordinal', title='Ano de Determinação'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determiner Name', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['determiner_full_name', 'det_year', 'counts'])
)

# g1.save('./graphs/determiner/counts_per_determiner.html')

# g1

<font color='red' size='5'>mesmo gráfico, ordenando também pela ordem</font>

In [7]:
teste1 = decapoda.groupby(['determiner_full_name','det_year','family']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [8]:
g = alt.Chart(teste1, width=800, height=1000, 
              title='').mark_circle().encode(
    x= alt.X('det_year', type='ordinal', title='Ano de Determinação'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determinador', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts:Q', title='Contagem',
                   legend= alt.Legend(columns=1),
                   scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('family:N', title="Família", 
                     scale=alt.Scale(domain=list(cores_familia_naive.keys()), 
                                     range=list(cores_familia_naive.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 102)),
    tooltip= alt.Tooltip(['determiner_full_name', 'det_year', 'counts', 'family'])
)

# configuring graphic elements (font, fontSize, etc)
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/determiner/counts_per_determiner-w-family.html')

# g

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [9]:
# ordenando
teste1.sort_values(['det_year', 'determiner_full_name'], inplace=True)

# salvando ordem das entradas
sorting = list(teste1['determiner_full_name'].unique())

In [10]:
g = alt.Chart(teste1, width=500, height=1000,
              title='').mark_circle().encode(
    x= alt.X('det_year', type='ordinal', title='Ano de Determinação'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determinador', 
            sort=sorting),
    size= alt.Size('counts', title='Contagem',type="quantitative",
                   scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('family:N', title="Família", 
                     scale=alt.Scale(domain=list(cores_familia_naive.keys()), 
                                     range=list(cores_familia_naive.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 102)),
    tooltip= alt.Tooltip(['determiner_full_name', 'det_year', 'counts', 'family'])
)

# configuring graphic elements (font, fontSize, etc)
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/determiner/counts_per_determiner-w-family-reordenado.html')

# g

<br>

### Gráfico dos determinadores mais expressivos 

<font color='red' size='5'>mesmo gráfico, ordenado pela soma das contagens </font>

In [11]:
# somando contagens de cada pesquisador
sorting = teste1.groupby('determiner_full_name').sum()['counts'].reset_index().rename(
    columns={'counts':'sum'})

sorting = sorting.sort_values('sum', ascending=False)

# Nomes dos pesquisadores ordenados
sort_list = sorting['determiner_full_name'].unique()

In [14]:
g = alt.Chart(teste1[teste1['determiner_full_name'].isin(sort_list[:50])], width=500, height=1000,
              title='').mark_circle().encode(
    x= alt.X('det_year', type='ordinal', title='Ano de Determinação'),
    y= alt.Y('determiner_full_name', type='nominal', title='Determinador', 
            sort=sort_list[:50]),
    size= alt.Size('counts:Q', title='Contagem', legend=alt.Legend(columns=1),
                   scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('family:N', title="Família", 
                     scale=alt.Scale(domain=list(cores_familia_naive.keys()), 
                                     range=list(cores_familia_naive.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 102)),
    tooltip= alt.Tooltip(['determiner_full_name', 'det_year', 'counts', 'family'])
)

# configuring graphic elements (font, fontSize, etc)
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/determiner/counts_per_determiner-w-family-ordenado_pela_soma.html')

# g

<br>

### Creating chart: counts per collector per year

<font color='red' size='5'> Collectors name is sensitive data. Do not publish it without curator's permission </font>

In [15]:
teste = decapoda.groupby(['collector_full_name','start_year']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [16]:
g1 = alt.Chart(teste, width=500, height=800).mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Ano de Coleta'),
    y= alt.Y('collector_full_name', type='nominal', title='Coletor', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts', scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts'])
)


# configuring graphic elements (font, fontSize, etc)
g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/collector/counts_per_collector.html')

# g1

<font color='red' size='5'>mesmo gráfico, colorindo por família</font>

**OBS:** Note que

- as contagens mudam (porque estamos ordenando por um fator a mais)
- há pontos sobrepostos (semelhante ao que tinhamos para a base crustaceas - para um mesmo ano, um mesmo pesquisador descobriu animais de ordens/familias diferentes)

In [17]:
teste1 = NewTable.groupby(['collector_full_name','start_year', 'family']).count()['class'].reset_index().rename(columns=
                                                                                            {'class':'counts'})

In [18]:
g = alt.Chart(teste1, width=500, height=800,
              title='').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Ano de Coleta'),
    y= alt.Y('collector_full_name', type='nominal', title='Coletor', 
            sort=alt.EncodingSortField('counts', op="count", order='descending')),
    size= alt.Size('counts:Q', title='Contagem',scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('family:N', title="Família", 
                     scale=alt.Scale(domain=list(cores_familia_naive.keys()), 
                                     range=list(cores_familia_naive.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 102)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

# configuring graphic elements (font, fontSize, etc)
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/collector/counts_per_collector-w-family.html')

# g

<br>

<font color='red'>**mesmo gráfico, agora ordenando pelo primeiro ano que o pesquisador aparece na base**</font>

In [19]:
# ordenando
teste1.sort_values(['start_year', 'collector_full_name'], inplace=True)

# salvando ordem das entradas
sorting = list(teste1['collector_full_name'].unique())

In [20]:
g = alt.Chart(teste1, width=500, height=800,
              title='').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Ano de Coleta'),
    y= alt.Y('collector_full_name', type='nominal', title='Coletor', 
            sort=alt.EncodingSortField('start_year', op="min", order='ascending')),
    size= alt.Size('counts:Q', title='Contagem',scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    order= alt.Order('counts', sort='descending'),  # smaller points in front    
    color= alt.Color('family:N', title="Família", 
                     scale=alt.Scale(domain=list(cores_familia_naive.keys()), 
                                     range=list(cores_familia_naive.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 102)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

# configuring graphic elements (font, fontSize, etc)
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/collector/counts_per_collector-w-family-reordenado.html')

# g

<br>

### Gráfico dos coletores mais expressivos

In [21]:
# somando contagens de cada pesquisador
sorting = teste1.groupby('collector_full_name').sum()['counts'].reset_index().rename(
    columns={'counts':'sum'})

sorting = sorting.sort_values('sum', ascending=False)

# Nomes dos pesquisadores ordenados
sort_list = sorting['collector_full_name'].unique()

In [23]:
g = alt.Chart(teste1[teste1['collector_full_name'].isin(sort_list[:50])], width=500, height=800,
              title='').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Ano de Coleta'),
    y= alt.Y('collector_full_name', type='nominal', title='Coletor', 
            sort=sort_list[:50]),
    size= alt.Size('counts:Q', title='Contagem',scale=alt.Scale(range=[15, 500])),  # range ajusta tamanho do circulo
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color= alt.Color('family:N', title="Família", 
                     scale=alt.Scale(domain=list(cores_familia_naive.keys()), 
                                     range=list(cores_familia_naive.values())),
                     legend= alt.Legend(columns=2, symbolLimit= 102)),
    tooltip= alt.Tooltip(['collector_full_name', 'start_year', 'counts', 'family'])
)

# configuring graphic elements (font, fontSize, etc)
g = g.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g.save('./graphs/collector/counts_per_collector-ordenado_pela_soma.html')

# g

<br>

**The end!**

-----